In [0]:
# MAGIC %md
# MAGIC # NYC Taxi Pipeline - Bronze to Silver ETL
# MAGIC ### Stack Tecnologias - Desafio Técnico
# MAGIC 
# MAGIC **Objetivo**: Transformar dados raw (Bronze) em dados limpos e padronizados (Silver)
# MAGIC 
# MAGIC **Transformações:**
# MAGIC 1. Limpeza de dados inválidos
# MAGIC 2. Conversão de tipos
# MAGIC 3. Enriquecimento com campos derivados
# MAGIC 4. Validação de qualidade

In [0]:
# COMMAND ----------
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Configurações - usando Unity Catalog
catalog_name = "nyc_taxi_catalog"
bronze_schema = "bronze"
silver_schema = "silver"

# Paths específicos
bronze_path = "s3://nyc-taxi-bronze-lucas/raw/"
silver_path = "s3://nyc-taxi-silver-lucas/processed/"

# COMMAND ----------
# Usar o catálogo correto
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {silver_schema}")

# COMMAND ----------
# Função para analisar filtros individualmente
def analyze_data_quality():
    """
    Analisar qualidade dos dados e impacto de cada filtro
    """
    print("=== ANÁLISE DE QUALIDADE DOS DADOS ===")
    
    df_bronze = spark.read.format("csv").option("header", "true").load(bronze_path)
    
    # Converter tipos básicos para análise
    df_typed = df_bronze.select(
        to_timestamp(col("tpep_pickup_datetime")).alias("pickup_datetime"),
        to_timestamp(col("tpep_dropoff_datetime")).alias("dropoff_datetime"),
        col("pickup_longitude").cast(DoubleType()),
        col("pickup_latitude").cast(DoubleType()),
        col("dropoff_longitude").cast(DoubleType()),
        col("dropoff_latitude").cast(DoubleType()),
        col("fare_amount").cast(DoubleType()),
        col("total_amount").cast(DoubleType()),
        col("trip_distance").cast(DoubleType()),
        col("passenger_count").cast(IntegerType())
    )
    
    total_records = df_typed.count()
    print(f"Total de registros: {total_records:,}")
    
    # Análise de cada filtro
    filters_analysis = [
        ("Timestamps não nulos", 
         df_typed.filter(col("pickup_datetime").isNotNull() & col("dropoff_datetime").isNotNull())),
        
        ("Duração positiva", 
         df_typed.filter(col("dropoff_datetime") > col("pickup_datetime"))),
        
        ("Coordenadas não nulas", 
         df_typed.filter(col("pickup_longitude").isNotNull() & col("pickup_latitude").isNotNull() & 
                        col("dropoff_longitude").isNotNull() & col("dropoff_latitude").isNotNull())),
        
        ("Coordenadas NYC válidas", 
         df_typed.filter((col("pickup_longitude").between(-74.5, -73.5)) &
                        (col("pickup_latitude").between(40.5, 41.0)) &
                        (col("dropoff_longitude").between(-74.5, -73.5)) &
                        (col("dropoff_latitude").between(40.5, 41.0)))),
        
        ("Valores monetários positivos", 
         df_typed.filter((col("fare_amount") >= 0) & (col("total_amount") > 0))),
        
        ("Passageiros válidos (1-6)", 
         df_typed.filter((col("passenger_count") >= 1) & (col("passenger_count") <= 6)))
    ]
    
    for filter_name, filtered_df in filters_analysis:
        count = filtered_df.count()
        percentage = (count / total_records) * 100
        print(f"{filter_name}: {count:,} ({percentage:.2f}%)")
    
    return df_typed

# Executar análise
df_analysis = analyze_data_quality()

# COMMAND ----------
def clean_and_transform_data():
    """
    Pipeline completo Bronze → Silver com filtros ajustados
    """
    # 1. Ler dados Bronze
    df_bronze = spark.read.format("csv")\
        .option("header", "true")\
        .load(bronze_path)
    
    # 2. Converter tipos e aplicar limpezas
    df_clean = df_bronze.select(
        # Timestamps
        to_timestamp(col("tpep_pickup_datetime")).alias("pickup_datetime"),
        to_timestamp(col("tpep_dropoff_datetime")).alias("dropoff_datetime"),
        
        # IDs e Flags
        col("VendorID").cast(IntegerType()).alias("vendor_id"),
        col("RateCodeID").cast(IntegerType()).alias("rate_code_id"),
        col("payment_type").cast(IntegerType()),
        col("store_and_fwd_flag"),
        
        # Passageiros e Distância
        col("passenger_count").cast(IntegerType()),
        col("trip_distance").cast(DoubleType()),
        
        # Coordenadas
        col("pickup_longitude").cast(DoubleType()),
        col("pickup_latitude").cast(DoubleType()),
        col("dropoff_longitude").cast(DoubleType()),
        col("dropoff_latitude").cast(DoubleType()),
        
        # Valores monetários
        col("fare_amount").cast(DoubleType()),
        col("extra").cast(DoubleType()),
        col("mta_tax").cast(DoubleType()),
        col("tip_amount").cast(DoubleType()),
        col("tolls_amount").cast(DoubleType()),
        col("improvement_surcharge").cast(DoubleType()),
        col("total_amount").cast(DoubleType())
    )
    
    # 3. Aplicar filtros de qualidade AJUSTADOS
    df_filtered = df_clean.filter(
        # Filtros essenciais apenas
        (col("pickup_datetime").isNotNull()) &
        (col("dropoff_datetime").isNotNull()) &
        (col("dropoff_datetime") > col("pickup_datetime")) &
        
        # Coordenadas não nulas (sem validação de range muito restritiva)
        (col("pickup_longitude").isNotNull()) &
        (col("pickup_latitude").isNotNull()) &
        (col("dropoff_longitude").isNotNull()) &
        (col("dropoff_latitude").isNotNull()) &
        
        # Coordenadas em range mais amplo para NYC (incluindo aeroportos e áreas próximas)
        (col("pickup_longitude").between(-75.0, -73.0)) &
        (col("pickup_latitude").between(40.0, 41.5)) &
        (col("dropoff_longitude").between(-75.0, -73.0)) &
        (col("dropoff_latitude").between(40.0, 41.5)) &
        
        # Valores monetários básicos
        (col("fare_amount").isNotNull()) &
        (col("total_amount").isNotNull()) &
        (col("total_amount") > 0) &
        
        # Filtros mais relaxados
        (col("trip_distance").isNotNull()) &
        (col("trip_distance") >= 0) &
        (col("trip_distance") <= 1000) &  # Aumentado para 1000 milhas
        
        # Passageiros mais flexível
        (col("passenger_count").isNotNull()) &
        (col("passenger_count") >= 0) &  # Permitir 0 passageiros
        (col("passenger_count") <= 9)    # Aumentado para 9
    )
    
    # 4. Enriquecer com campos derivados
    df_enriched = df_filtered.select(
        "*",
        # Duração da viagem
        ((unix_timestamp("dropoff_datetime") - unix_timestamp("pickup_datetime"))/60).cast(IntegerType()).alias("trip_duration_minutes"),
        
        # Campos temporais
        hour("pickup_datetime").alias("pickup_hour"),
        dayofweek("pickup_datetime").alias("pickup_dayofweek"),
        month("pickup_datetime").alias("pickup_month"),
        
        # Distância calculada (Haversine) - versão simplificada
        (111.0 * sqrt(
            pow(col("dropoff_latitude") - col("pickup_latitude"), 2) +
            pow(cos(radians(col("pickup_latitude"))) * (col("dropoff_longitude") - col("pickup_longitude")), 2)
        )).alias("calculated_distance_km"),
        
        # Mapeamento de payment_type
        when(col("payment_type") == 1, "Credit card")
        .when(col("payment_type") == 2, "Cash")
        .when(col("payment_type") == 3, "No charge")
        .when(col("payment_type") == 4, "Dispute")
        .when(col("payment_type") == 5, "Unknown")
        .otherwise("Other").alias("payment_type_desc"),
        
        # Flags de qualidade para rastreamento
        when((col("fare_amount") < 0) | (col("total_amount") < col("fare_amount")), "warning")
        .otherwise("valid").alias("quality_flag"),
        
        # Metadados
        current_timestamp().alias("processed_timestamp"),
        lit("bronze_to_silver_etl").alias("processing_stage")
    )
    
    return df_enriched

# COMMAND ----------
# Executar transformação
print("=== EXECUTANDO TRANSFORMAÇÃO BRONZE → SILVER ===")
df_silver = clean_and_transform_data()

# Validar resultado
bronze_count = spark.read.csv(bronze_path, header=True).count()
silver_count = df_silver.count()

print(f"Registros Bronze: {bronze_count:,}")
print(f"Registros Silver: {silver_count:,}")
print(f"Taxa de retenção: {(silver_count / bronze_count) * 100:.2f}%")

# COMMAND ----------
# Análise de qualidade dos dados processados
print("=== ANÁLISE DE QUALIDADE SILVER ===")

quality_analysis = df_silver.groupBy("quality_flag").count().collect()
for row in quality_analysis:
    print(f"Records com {row['quality_flag']}: {row['count']:,}")

# Estatísticas básicas
df_silver.select(
    avg("trip_duration_minutes").alias("avg_duration_min"),
    avg("calculated_distance_km").alias("avg_distance_km"),
    avg("fare_amount").alias("avg_fare"),
    min("pickup_datetime").alias("min_date"),
    max("pickup_datetime").alias("max_date")
).show()

# COMMAND ----------
# Salvar como tabela Delta no Unity Catalog
table_name = f"{catalog_name}.{silver_schema}.nyc_taxi_trips"

df_silver.write\
    .format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "true")\
    .option("path", silver_path)\
    .saveAsTable(table_name)

print(f"✅ Dados salvos na tabela {table_name} com sucesso!")

# COMMAND ----------
# Validação final
print("=== VALIDAÇÃO FINAL ===")

spark.sql(f"SELECT COUNT(*) as total_records FROM {table_name}").show()

spark.sql(f"""
SELECT 
    pickup_hour,
    payment_type_desc,
    COUNT(*) as trip_count,
    AVG(trip_duration_minutes) as avg_duration,
    AVG(calculated_distance_km) as avg_distance,
    AVG(fare_amount) as avg_fare
FROM {table_name} 
GROUP BY pickup_hour, payment_type_desc 
ORDER BY pickup_hour, trip_count DESC
LIMIT 20
""").show()

# COMMAND ----------
# MAGIC %md
# MAGIC ## Resumo da Transformação
# MAGIC 
# MAGIC 1. **Filtros Aplicados:**
# MAGIC    - Timestamps válidos e duração positiva
# MAGIC    - Coordenadas em range amplo de NYC
# MAGIC    - Valores monetários não nulos e positivos
# MAGIC    - Distância e passageiros em ranges realistas
# MAGIC 
# MAGIC 2. **Enriquecimentos:**
# MAGIC    - Duração da viagem em minutos
# MAGIC    - Campos temporais (hora, dia da semana, mês)
# MAGIC    - Distância calculada usando coordenadas
# MAGIC    - Mapeamento de códigos de pagamento
# MAGIC    - Flags de qualidade para monitoramento
# MAGIC 
# MAGIC 3. **Próximos Passos:**
# MAGIC    - Implementar agregações para camada Gold
# MAGIC    - Configurar monitoramento de qualidade
# MAGIC    - Otimizar particionamento para performance

=== ANÁLISE DE QUALIDADE DOS DADOS ===
Total de registros: 47,248,845
Timestamps não nulos: 47,248,845 (100.00%)
Duração positiva: 47,197,465 (99.89%)
Coordenadas não nulas: 47,248,845 (100.00%)
Coordenadas NYC válidas: 46,393,821 (98.19%)
Valores monetários positivos: 47,227,344 (99.95%)
Passageiros válidos (1-6): 47,240,404 (99.98%)
=== EXECUTANDO TRANSFORMAÇÃO BRONZE → SILVER ===
Registros Bronze: 47,248,845
Registros Silver: 46,385,374
Taxa de retenção: 98.17%
=== ANÁLISE DE QUALIDADE SILVER ===
Records com valid: 46,385,364
Records com warning: 10
+------------------+------------------+------------------+-------------------+-------------------+
|  avg_duration_min|   avg_distance_km|          avg_fare|           min_date|           max_date|
+------------------+------------------+------------------+-------------------+-------------------+
|14.485172136372125|3.3347917296700875|12.364176056012825|2015-01-01 00:00:00|2016-03-31 23:59:59|
+------------------+------------------+------